# Pairs detected by cointegration 

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

from statsmodels.tsa.stattools import coint
# from tqdm import tqdm
from tqdm import notebook

# Find and replace: Ctrl + Shift + h


In [ ]:
a = list(combinations([1,2,3], 2))
a[0][0]

1

### Functionality

In [ ]:
def find_cointegrated_pairs(data: pd.DataFrame,
                            tag: str,
                            significance: float = 0.01) -> tuple:
    """Function calculating cointegration measures for the given data"""
    
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    
    for i in notebook.tqdm(range(n)):
        
        for j in range(i+1, n):
            
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            
            pair_string = keys[i] + keys[j]
            
            if (pvalue < significance) and pair_string.count(tag) == 1:
                pairs.append((keys[i], keys[j]))
                
                print(f'{keys[i]} | {keys[j]} | {pvalue:.3f}')
                
    return score_matrix, pvalue_matrix, pairs


def CHECK_COINT(data: pd.DataFrame,
                tag: str,
                significance: float = 0.01) -> dict:
    """Function for calculating and returning cointegrated pairs"""
    
    n = data.shape[1]
    keys = data.columns
    pairs = {}
    
    for i in notebook.tqdm(range(n)):
        
        for j in notebook.tqdm(range(i+1, n)):
            
            S1 = data[keys[i]]
            S2 = data[keys[j]]

            _, p_value, _ = coint(S1, S2)
            
            pair_string = keys[i] + keys[j]
            
            if (p_value < significance) and pair_string.count(tag) == 1:

                pair_name = keys[i].replace(tag, '') + ' - ' + keys[j].replace(tag, '')
                pairs[pair_name] = round(p_value, 3)
                
                print(f'{pair_name} | {p_value:.3f}')
                
    return pairs

def CHECK_COINT_2(data: pd.DataFrame,
                  tag: str,
                  significance: float = 0.01) -> dict:
    """Function for calculating and returning cointegrated pairs"""
    
    all_pairs = list(combinations(data.columns, 2))
    relevant_pairs = [p for p in all_pairs if (p[0] + p[1]).count(tag) == 1]

    pairs_ = {}

    # Only for the purpose of displaying the progress bar
    for i in notebook.tqdm(range(len(relevant_pairs))):
    # for p_ in releavnt_pairs:
        p_ = relevant_pairs[i]

        S1 = data[p_[0]]
        S2 = data[p_[1]]

        _, p_value, _ = coint(S1, S2)

        if (p_value < significance):

            pair_name = p_[0].replace(tag, '') + ' - ' + p_[1].replace(tag, '')
            pairs_[pair_name] = p_value

            print(pair_name)

    return pairs_


## Country
For each of the foreign countries, the norwegain equities will be concatenated, and the resulting dataframe will be fed to the cointegration-test.

The test will only extract pairs where there are on foreign equity and one norwegian equity. This is done by adding a tag to the foreign column names, and filtering out pairs where the number of foreign equities is either zero or two

### Norway

In [ ]:
equities_obx = pd.read_excel('EQUITIES.xlsx', sheet_name='Norway', index_col='Name')
equities_obx.head()

,EQUINOR,DNB BANK,TELENOR,NORSK HYDRO,AKER BP,MOWI,YARA INTERNATIONAL,AKER,DNO,GJENSIDIGE FORSIKRING,GRIEG SEAFOOD,ORKLA,SALMAR,SCHIBSTED A,TOMRA SYSTEMS,AF GRUPPEN 'A',ARENDALS FOSSEKOMPANI,ATEA,AUSTEVOLL SEAFOOD,BAKKAFROST,BONHEUR,BORREGAARD,EIENDOMSSPAR,FRONTLINE,GOLAR LNG (NAS),KONGSBERG GRUPPEN,LEROY SEAFOOD GROUP,NEL,NORDIC SEMICONDUCTOR,NORWAY ROYAL SALMON,NORWEGIAN AIR SHUTTLE,PROTECTOR FORSIKRING,STOREBRAND,SUBSEA 7,VEIDEKKE,WALLENIUS WILHELMSEN
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-02,338.21,1879.65,376.75,10737.69,153.12,3.54,639.17,522.51,192.56,159.65,58.60,30689.68,135.37,1418.65,383.46,1948.42,1333.49,73.85,83.35,195.70,86240.19,103.86,76.0,240.88,975.05,2164.81,594.02,1.24,2453.80,70.35,532.74,118.50,529.19,261.94,5014.19,238.12
2013-01-03,337.97,1879.65,379.08,10800.80,153.66,3.52,641.71,527.30,192.16,159.65,57.67,30590.83,134.49,1423.35,372.35,1965.66,1333.49,73.24,82.78,191.67,86854.00,101.93,76.0,247.25,961.61,2138.41,594.02,1.23,2422.54,71.55,542.89,122.47,533.95,261.17,4969.32,236.94
2013-01-04,339.16,1905.76,381.41,10693.13,150.00,3.48,640.79,526.10,192.96,158.48,58.14,30572.30,135.37,1427.47,369.76,1948.42,1333.49,73.54,83.07,192.48,86240.19,103.86,76.0,262.54,964.14,2103.21,611.75,1.22,2500.69,71.55,552.70,121.59,548.22,265.02,4991.75,239.29
2013-01-07,336.30,1905.76,380.41,10600.30,150.00,3.43,633.17,526.10,190.46,157.70,58.14,30300.49,135.37,1424.53,370.50,1957.04,1333.49,73.24,82.50,188.45,85626.38,104.83,76.0,253.62,955.77,2094.41,611.75,1.24,2516.31,75.40,547.96,121.59,544.42,261.17,5002.97,239.29
2013-01-08,336.06,1909.67,382.41,10566.89,150.45,3.44,639.63,526.10,194.56,159.16,58.14,30362.27,135.37,1424.53,363.09,1965.66,1333.49,73.54,82.50,186.84,86547.06,104.35,76.0,252.35,956.02,2085.61,620.62,1.28,2531.94,75.40,551.01,122.47,546.32,260.98,5014.19,242.81


### France

In [ ]:
equities_france = pd.read_excel('EQUITIES.xlsx', sheet_name='France', index_col='Name')
equities_france.columns = [c + 'FRANCE' for c in equities_france.columns]
equities_france_obx = pd.concat([equities_france, equities_obx], axis=1)
equities_france_obx_returns = equities_france_obx.pct_change()[1:]
equities_france_obx_returns.head()

,LVMHFRANCE,L'OREALFRANCE,TOTALENERGIESFRANCE,HERMES INTL.FRANCE,SANOFIFRANCE,CHRISTIAN DIORFRANCE,AIRBUSFRANCE,BNP PARIBASFRANCE,KERINGFRANCE,SCHNEIDER ELECTRICFRANCE,ESSILORLUXOTTICAFRANCE,L AIR LQE.SC.ANYME. POUR L ETUDE ET L EPXTN.FRANCE,AXAFRANCE,VINCIFRANCE,DASSAULT SYSTEMESFRANCE,PERNOD-RICARDFRANCE,SAFRANFRANCE,CREDIT AGRICOLEFRANCE,DANONEFRANCE,ENGIEFRANCE,SOCIETE GENERALEFRANCE,ORANGEFRANCE,STMICROELECTRONICSFRANCE,CAPGEMINIFRANCE,SARTORIUS STEDIM BIOTECHFRANCE,SAINT GOBAINFRANCE,CMPG.DES ETS.MICH.FRANCE,EDFFRANCE,VEOLIA ENVIRONFRANCE,LEGRANDFRANCE,CARREFOURFRANCE,CNP ASSURANCESFRANCE,EUROFINS SCIEN.FRANCE,PUBLICIS GROUPEFRANCE,RENAULTFRANCE,TELEPERFORMANCEFRANCE,THALESFRANCE,ACCORFRANCE,ADPFRANCE,BOLLOREFRANCE,...,SODEXOFRANCE,VALEOFRANCE,VIVENDIFRANCE,AIR FRANCE-KLMFRANCE,EQUINOR,DNB BANK,TELENOR,NORSK HYDRO,AKER BP,MOWI,YARA INTERNATIONAL,AKER,DNO,GJENSIDIGE FORSIKRING,GRIEG SEAFOOD,ORKLA,SALMAR,SCHIBSTED A,TOMRA SYSTEMS,AF GRUPPEN 'A',ARENDALS FOSSEKOMPANI,ATEA,AUSTEVOLL SEAFOOD,BAKKAFROST,BONHEUR,BORREGAARD,EIENDOMSSPAR,FRONTLINE,GOLAR LNG (NAS),KONGSBERG GRUPPEN,LEROY SEAFOOD GROUP,NEL,NORDIC SEMICONDUCTOR,NORWAY ROYAL SALMON,NORWEGIAN AIR SHUTTLE,PROTECTOR FORSIKRING,STOREBRAND,SUBSEA 7,VEIDEKKE,WALLENIUS WILHELMSEN
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-03,-0.010563,-0.023019,-0.006544,-0.004207,-0.006146,0.006950,0.000953,-0.003553,0.012507,-0.007772,-0.008709,-0.010403,-0.002471,0.000770,-0.006104,-0.006153,-0.004602,-0.004012,-0.013980,0.004350,-0.012733,-0.008178,0.009300,-0.016242,-0.004476,-0.009356,-0.003282,-0.005961,-0.022243,0.003665,0.004171,0.002304,0.005766,0.001959,0.011069,-0.004975,-0.001275,-0.006136,-0.003553,0.007108,...,0.001741,-0.006399,-0.020986,0.047323,-0.000710,0.000000,0.006184,0.005877,0.003527,-0.005650,0.003974,0.009167,-0.002077,0.000000,-0.015870,-0.003221,-0.006501,0.003313,-0.028973,0.008848,0.0,-0.008260,-0.006839,-0.020593,0.007117,-0.018583,0.0,0.026445,-0.013784,-0.012195,0.000000,-0.008065,-0.012739,0.017058,0.019052,0.033502,0.008995,-0.002940,-0.008949,-0.004955
2013-01-04,0.005993,0.003403,0.000149,0.013314,0.012852,-0.003444,0.020998,-0.000667,0.002755,0.000674,-0.001315,0.002079,0.003925,0.004277,-0.007266,0.003533,-0.000327,0.001952,-0.001368,0.004518,-0.007274,0.005696,-0.013301,-0.009049,0.008865,0.004035,0.004702,0.018694,-0.000084,0.012654,0.019907,0.012753,0.001803,0.006643,-0.008463,0.002459,0.009593,0.011069,-0.000786,0.017492,...,0.004762,-0.007316,-0.017962,0.011736,0.003521,0.013891,0.006146,-0.009969,-0.023819,-0.011364,-0.001434,-0.002276,0.004163,-0.007329,0.008150,-0.000606,0.006543,0.002895,-0.006956,-0.008771,0.0,0.004096,0.003503,0.004226,-0.007067,0.018935,0.0,0.061840,0.002631,-0.016461,0.029847,-0.008130,0.032260,0.000000,0.018070,-0.007185,0.026725,0.014741,0.004514,0.009918
2013-01-07,-0.011037,-0.012779,-0.015757,0.005294,-0.002981,-0.002038,-0.000600,0.019569,-0.010067,-0.004895,-0.019263,-0.007326,0.003847,-0.008610,-0.000682,-0.004833,-0.007790,0.036307,-0.007248,-0.018116,0.028270,-0.009793,0.025351,0.017040,0.031540,-0.008932,-0.012455,-0.024737,-0.014409,-0.008850,-0.017812,0.016665,0.011771,-0.002800,-0.006150,-0.005642,-0.000739,-0.000885,-0.008714,0.001145,...,-0.000280,0.005036,-0.021378,0.036787,-0.008433,0.000000,-0.002622,-0.008681,0.000000,-0.014368,-0.011892,0.000000,-0.012956,-0.004922,0.000000,-0.008891,0.000000,-0.002060,0.002001,0.004424,0.0,-0.004079,-0.006862,-0.020937,-0.007117,0.009339,0.0,-0.033976,-0.008681,-0.004184,0.000000,0.016393,0.006246,0.053809,-0.008576,0.000000,-0.006932,-0.014527,0.002248,0.000000
2013-01-08,-0.002697,0.005556,0.008559,-0.000358,0.007725,-0.002978,0.008405,0.013415,0.007206,0.003026,0.015074,-0.000614,-0.021126,-0.000064,0.002669,0.003813,0.003127,0.010847,0.005399,0.010988,0.034443,-0.000288,0.026347,-0.010654,0.031113,-0.025215,-0.009706,0.007218,0.004482,-0.002681,0.000429,0.029147,0.019509,0.003316,-0.017421,-0.001647,0.003995,-0.00

In [ ]:
france_pairs = CHECK_COINT_2(equities_france_obx, tag = 'FRANCE')

LVMH - TOMRA SYSTEMS
L'OREAL - TOMRA SYSTEMS
TOTALENERGIES - AUSTEVOLL SEAFOOD
HERMES INTL. - BORREGAARD
AIRBUS - TELENOR
VINCI - MOWI
VINCI - ORKLA
DASSAULT SYSTEMES - SALMAR
PERNOD-RICARD - SALMAR
SAFRAN - TELENOR
ENGIE - ORKLA
ENGIE - TOMRA SYSTEMS
ENGIE - BAKKAFROST
ENGIE - FRONTLINE
ENGIE - LEROY SEAFOOD GROUP
CAPGEMINI - YARA INTERNATIONAL
TELEPERFORMANCE - TOMRA SYSTEMS
EIFFAGE - ORKLA
BUREAU VERITAS - DNB BANK
BUREAU VERITAS - ATEA
EDENRED - NORWEGIAN AIR SHUTTLE
GECINA - EIENDOMSSPAR
REMY COINTREAU - DNB BANK
VIVENDI - BAKKAFROST


In [ ]:
CHECK_COINT(equities_france_obx, tag='FRANCE')

### US

In [ ]:
equities_US = pd.read_excel('EQUITIES.xlsx', sheet_name='US', index_col='Name')
equities_US.columns = [c + 'US' for c in equities_US.columns]
equities_US_obx = pd.concat([equities_US, equities_obx], axis=1)
equities_US_obx.head()

,APPLEUS,MICROSOFTUS,AMAZON.COMUS,TESLAUS,ALPHABET AUS,NVIDIAUS,META PLATFORMS AUS,JP MORGAN CHASE & CO.US,JOHNSON & JOHNSONUS,UNITEDHEALTH GROUPUS,PROCTER & GAMBLEUS,BANK OF AMERICAUS,WALMARTUS,VISA 'A'US,MASTERCARDUS,EXXON MOBILUS,HOME DEPOTUS,BERKSHIRE HATHAWAY 'A'US,COCA COLAUS,WALT DISNEYUS,PFIZERUS,CHEVRONUS,DYCOM INDS.US,ABBVIEUS,BROADCOMUS,ELI LILLYUS,VERIZON COMMUNICATIONSUS,CISCO SYSTEMSUS,PEPSICOUS,ADOBE (NAS)US,COSTCO WHOLESALEUS,SALESFORCE.COMUS,ABBOTT LABORATORIESUS,ACCENTURE CLASS AUS,COMCAST AUS,INTELUS,ORACLEUS,WELLS FARGO & COUS,THERMO FISHER SCIENTIFICUS,ADVANCED MICRO DEVICESUS,...,T-MOBILE USUS,TEXAS INSTRUMENTSUS,UNION PACIFICUS,UNITED PARCEL SER.'B'US,EQUINOR,DNB BANK,TELENOR,NORSK HYDRO,AKER BP,MOWI,YARA INTERNATIONAL,AKER,DNO,GJENSIDIGE FORSIKRING,GRIEG SEAFOOD,ORKLA,SALMAR,SCHIBSTED A,TOMRA SYSTEMS,AF GRUPPEN 'A',ARENDALS FOSSEKOMPANI,ATEA,AUSTEVOLL SEAFOOD,BAKKAFROST,BONHEUR,BORREGAARD,EIENDOMSSPAR,FRONTLINE,GOLAR LNG (NAS),KONGSBERG GRUPPEN,LEROY SEAFOOD GROUP,NEL,NORDIC SEMICONDUCTOR,NORWAY ROYAL SALMON,NORWEGIAN AIR SHUTTLE,PROTECTOR FORSIKRING,STOREBRAND,SUBSEA 7,VEIDEKKE,WALLENIUS WILHELMSEN
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-02,93192.926112,206245.074024,72496.960272,816.659976,3977.306784,4305.879864,404.109024,23177.947848,34005.079152,219362.395560,33734.330520,6414.706584,805124.32968,1567.660512,6238.143384,110279.333208,1.319698e+06,1.149719e+06,42886.870224,18465.751920,50312.787360,74129.231880,74450.245848,553.635984,1146.502104,19031.030040,12701.625552,150820.064736,66923.963568,66709.770336,6956.369376,5490.949992,182573.852736,2853.592368,513180.609744,363999.548088,313682.070768,64239.982224,27893.564688,3243.134928,...,195.323040,15778.625544,72186.209040,821.349936,338.21,1879.65,376.75,10737.69,153.12,3.54,639.17,522.51,192.56,159.65,58.60,30689.68,135.37,1418.65,383.46,1948.42,1333.49,73.85,83.35,195.70,86240.19,103.86,76.0,240.88,975.05,2164.81,594.02,1.24,2453.80,70.35,532.74,118.50,529.19,261.94,5014.19,238.12
2013-01-03,92809.727360,205237.619500,73454.963496,809.959208,4013.900500,4346.476852,404.256128,23330.765352,34250.021664,210910.060904,33809.591736,6432.425116,806909.47840,1582.408968,6300.975092,111030.080856,1.327308e+06,1.167438e+06,43256.852648,18665.179932,50629.351348,74450.410820,74061.904208,553.793052,1162.458976,19276.628456,12750.429720,152943.883788,67530.529836,66249.866012,7088.339588,5458.737724,191158.665236,2867.803212,511829.718788,366109.464600,312922.514288,64258.080932,28047.606264,3219.356896,...,195.616780,15707.888304,72910.965196,829.214800,337.97,1879.65,379.08,10800.80,153.66,3.52,641.71,527.30,192.16,159.65,57.67,30590.83,134.49,1423.35,372.35,1965.66,1333.49,73.24,82.78,191.67,86854.00,101.93,76.0,247.25,961.61,2138.41,594.02,1.23,2422.54,71.55,542.89,122.47,533.95,261.17,4969.32,236.94
2013-01-04,90941.892654,202995.940182,74230.199892,807.697506,4125.769794,4525.551732,421.995162,23932.897758,34917.337026,212994.134742,34147.278594,6564.793008,816391.51506,1607.990604,6350.738304,112430.045610,1.335314e+06,1.178837e+06,43669.964316,19173.434046,51248.600280,75437.679336,75503.421504,551.179644,1164.174876,20148.740424,12921.701652,154398.959376,68164.980048,67448.210916,7121.638146,5531.429340,191518.602936,2906.510610,515623.788972,366247.879014,318165.560658,65103.818280,28519.031010,3375.232074,...,199.189794,15798.819006,74775.433572,837.876078,339.16,1905.76,381.41,10693.13,150.00,3.48,640.79,526.10,192.96,158.48,58.14,30572.30,135.37,1427.47,369.76,1948.42,1333.49,73.54,83.07,192.48,86240.19,103.86,76.0,262.54,964.14,2103.21,611.75,1.22,2500.69,71.55,552.70,121.59,548.22,265.02,4991.75,239.29
2013-01-07,90306.998528,202392.402752,76811.691328,805.403968,4103.223360,4389.883072,431.222272,23932.780064,34805.621568,212758.714976,33877.507520,6546.722848,807695.67680,1617.699872,6453.597664,111005.499488,1.326685e+06,1.172407e+06,43204.706304,18704.546208,51231.402368,74842

In [ ]:
US_pairs = CHECK_COINT_2(equities_US_obx, tag='US')

  0%|          | 0/2075 [00:00<?, ?it/s]

META PLATFORMS A - BAKKAFROST
JOHNSON & JOHNSON - AF GRUPPEN 'A'
JOHNSON & JOHNSON - BAKKAFROST
JOHNSON & JOHNSON - EIENDOMSSPAR
BANK OF AMERICA - DNB BANK
BANK OF AMERICA - ATEA
WALMART - NORWEGIAN AIR SHUTTLE
HOME DEPOT - GJENSIDIGE FORSIKRING
PFIZER - DNB BANK
PFIZER - AKER BP
PFIZER - MOWI
PFIZER - SALMAR
PFIZER - AF GRUPPEN 'A'
PFIZER - BAKKAFROST
PFIZER - LEROY SEAFOOD GROUP
BROADCOM - GJENSIDIGE FORSIKRING
VERIZON COMMUNICATIONS - EIENDOMSSPAR
CISCO SYSTEMS - MOWI
PEPSICO - BAKKAFROST
ADOBE (NAS) - TOMRA SYSTEMS
SALESFORCE.COM - TOMRA SYSTEMS
ABBOTT LABORATORIES - TOMRA SYSTEMS
COMCAST A - AF GRUPPEN 'A'
COMCAST A - BAKKAFROST
INTEL - BAKKAFROST
INTEL - NORWAY ROYAL SALMON
ORACLE - YARA INTERNATIONAL
ADVANCED MICRO DEVICES - BONHEUR
MCDONALDS - BAKKAFROST
NIKE 'B' - TOMRA SYSTEMS
CITIGROUP - TELENOR
AMERICAN EXPRESS - TOMRA SYSTEMS
CHARLES SCHWAB - DNB BANK
LINDE - TOMRA SYSTEMS
RAYTHEON TECHNOLOGIES - MOWI
UNITED PARCEL SER.'B' - BORREGAARD
AKER BP - ATEVOLL SEAFOOD
AKER - ATEV

IndexError: ignored

### UK

In [ ]:
equities_UK = pd.read_excel('EQUITIES.xlsx', sheet_name='UK', index_col='Name')
equities_UK.columns = [c + 'UK' for c in equities_UK.columns]
equities_UK_obx = pd.concat([equities_UK, equities_obx], axis=1)

UK_pairs = CHECK_COINT_2(equities_UK_obx, tag='UK')

  0%|          | 0/2088 [00:00<?, ?it/s]

DIAGEO - SALMAR
NATIONAL GRID - VEIDEKKE
LEGAL & GENERAL - YARA INTERNATIONAL
BURBERRY GROUP - DNB BANK
BURBERRY GROUP - TELENOR
OCADO GROUP - NEL
AVEVA GROUP - NORWEGIAN AIR SHUTTLE
BARRATT DEVELOPMENTS - BAKKAFROST
BUNZL - VEIDEKKE
MONDI - EIENDOMSSPAR
PERSIMMON - BAKKAFROST
RIGHTMOVE - BAKKAFROST
SAINSBURY J - NORDIC SEMICONDUCTOR


### Italy

In [ ]:
equities_Italy = pd.read_excel('EQUITIES.xlsx', sheet_name='Italy', index_col='Name')
equities_Italy.columns = [c + 'Italy' for c in equities_Italy.columns]
equities_Italy_obx = pd.concat([equities_Italy, equities_obx], axis=1)

Italy_pairs = CHECK_COINT_2(equities_Italy_obx, tag='Italy')

  0%|          | 0/1620 [00:00<?, ?it/s]

STELLANTIS - DNB BANK
STELLANTIS - ATEA
EXOR ORD - ATEA
SNAM - GJENSIDIGE FORSIKRING
SNAM - AF GRUPPEN 'A'
SNAM - BAKKAFROST
TERNA RETE ELETTRICA NAZ - GJENSIDIGE FORSIKRING
SAIPEM - ORKLA
SAIPEM - AF GRUPPEN 'A'
SAIPEM - EIENDOMSSPAR
SAIPEM - GOLAR LNG (NAS)
SAIPEM - NORWAY ROYAL SALMON
A2A - MOWI
AMPLIFON - TOMRA SYSTEMS
BANCA MONTE DEI PASCHI - EIENDOMSSPAR
BUZZI UNICEM - ORKLA
DIASORIN - NEL
MEDIOBANCA BC.FIN - ATEA
ACEA - BAKKAFROST
ASCOPIAVE - ORKLA
BANCA CARIGE - ORKLA
BANCA CARIGE - TOMRA SYSTEMS
BANCA CARIGE - BAKKAFROST
BANCA CARIGE - EIENDOMSSPAR
BANCA CARIGE - NEL
BANCA GENERALI - YARA INTERNATIONAL
BRUNELLO CUCINELLI - DNB BANK
BRUNELLO CUCINELLI - AKER


### Canada

In [ ]:
equities_Canada = pd.read_excel('EQUITIES.xlsx', sheet_name='Canada', index_col='Name')
equities_Canada.columns = [c + 'Canada' for c in equities_Canada.columns]
equities_Canada_obx = pd.concat([equities_Canada, equities_obx], axis=1)

Canada_pairs = CHECK_COINT_2(equities_Canada_obx, tag='Canada')

  0%|          | 0/1980 [00:00<?, ?it/s]

BROOKFIELD ASSET MAN.'A' (NYS) - GJENSIDIGE FORSIKRING
CANADIAN NATIONAL RY. - GJENSIDIGE FORSIKRING
BCE - AF GRUPPEN 'A'
ALIMENTATION COUCHE TARD MULTI VOTING A - AF GRUPPEN 'A'
MANULIFE FINANCIAL - ATEA
CONSTELLATION SOFTWARE - GJENSIDIGE FORSIKRING
CONSTELLATION SOFTWARE - TOMRA SYSTEMS
TELUS - GJENSIDIGE FORSIKRING
GREAT WEST LIFECO - YARA INTERNATIONAL
SUN LIFE FINL. - GJENSIDIGE FORSIKRING
AGNICO-EAGLE MNS. (NYS) - ORKLA
INTACT FINANCIAL - GJENSIDIGE FORSIKRING
POWER CORP.CANADA - BORREGAARD
ROGERS COMMS.'B' - MOWI
WHEATON PRECIOUS METALS - NEL
CGI - MOWI
CGI - BAKKAFROST
OPEN TEXT (NAS) - BAKKAFROST
PEMBINA PIPELINE - MOWI
PEMBINA PIPELINE - LEROY SEAFOOD GROUP
WESTON GEORGE - VEIDEKKE
CAE - MOWI
CAE - ATEA
METRO - AF GRUPPEN 'A'
METRO - BAKKAFROST
SHAW COMMS.'B' - YARA INTERNATIONAL


### Australia

In [ ]:
equities_Australia = pd.read_excel('EQUITIES.xlsx', sheet_name='Australia', index_col='Name')
equities_Australia.columns = [c + 'Australia' for c in equities_Australia.columns]
equities_Australia_obx = pd.concat([equities_Australia, equities_obx], axis=1)

Australia_pairs = CHECK_COINT_2(equities_Australia_obx, tag='Australia')

  0%|          | 0/1944 [00:00<?, ?it/s]

COMMONWEALTH BK.OF AUS. - YARA INTERNATIONAL
COMMONWEALTH BK.OF AUS. - BORREGAARD
NATIONAL AUS.BANK - NORSK HYDRO
MACQUARIE GROUP - SALMAR
WESFARMERS - TOMRA SYSTEMS
WOOLWORTHS GROUP - NORWEGIAN AIR SHUTTLE
GOODMAN GROUP - TOMRA SYSTEMS
TRANSURBAN GROUP STAPLED UNITS - ORKLA
NEWCREST MINING - NORWAY ROYAL SALMON
ARISTOCRAT LEISURE - ATEA
JAMES HARDIE INDUSTRIES CDI - BORREGAARD
MINERAL RESOURCES - BORREGAARD
QBE INSURANCE GROUP - EQUINOR
QBE INSURANCE GROUP - DNB BANK
QBE INSURANCE GROUP - TELENOR
QBE INSURANCE GROUP - NORSK HYDRO
QBE INSURANCE GROUP - AKER BP
QBE INSURANCE GROUP - MOWI
QBE INSURANCE GROUP - YARA INTERNATIONAL
QBE INSURANCE GROUP - AKER
QBE INSURANCE GROUP - GJENSIDIGE FORSIKRING
QBE INSURANCE GROUP - GRIEG SEAFOOD
QBE INSURANCE GROUP - ORKLA
QBE INSURANCE GROUP - SALMAR
QBE INSURANCE GROUP - SCHIBSTED A
QBE INSURANCE GROUP - TOMRA SYSTEMS
QBE INSURANCE GROUP - AF GRUPPEN 'A'
QBE INSURANCE GROUP - ARENDALS FOSSEKOMPANI
QBE INSURANCE GROUP - ATEA
QBE INSURANCE GROUP - A

### Austria

In [ ]:
equities_Austria = pd.read_excel('EQUITIES.xlsx', sheet_name='Austria', index_col='Name')
equities_Austria.columns = [c + 'Austria' for c in equities_Austria.columns]
equities_Austria_obx = pd.concat([equities_Austria, equities_obx], axis=1)

Austria_pairs = CHECK_COINT_2(equities_Austria_obx, tag='Austria')

  0%|          | 0/1368 [00:00<?, ?it/s]

WIENERBERGER - ATEA
EVN - DNB BANK
AT&S AU.TCHG.& SYSTK. - YARA INTERNATIONAL
OSTERREICHISCHE POST - DNB BANK
OSTERREICHISCHE POST - ATEA
OSTERREICHISCHE POST - STOREBRAND
S IMMO - MOWI
ATHOS IMMOBILIEN - MOWI
ATHOS IMMOBILIEN - AF GRUPPEN 'A'
ATHOS IMMOBILIEN - BAKKAFROST
ATHOS IMMOBILIEN - EIENDOMSSPAR
ATHOS IMMOBILIEN - LEROY SEAFOOD GROUP
BURGENLAND HOLDING - BAKKAFROST
JOSEF MANNER & CO. - NORWEGIAN AIR SHUTTLE
LINZ TEXTIL - SUBSEA 7
LINZ TEXTIL - WALLENIUS WILHELMSEN
MASCHINEN.HEID ST - EQUINOR
MASCHINEN.HEID ST - DNB BANK
MASCHINEN.HEID ST - TELENOR
MASCHINEN.HEID ST - NORSK HYDRO
MASCHINEN.HEID ST - AKER BP
MASCHINEN.HEID ST - MOWI
MASCHINEN.HEID ST - YARA INTERNATIONAL
MASCHINEN.HEID ST - AKER
MASCHINEN.HEID ST - DNO
MASCHINEN.HEID ST - GJENSIDIGE FORSIKRING
MASCHINEN.HEID ST - GRIEG SEAFOOD
MASCHINEN.HEID ST - ORKLA
MASCHINEN.HEID ST - SALMAR
MASCHINEN.HEID ST - SCHIBSTED A
MASCHINEN.HEID ST - TOMRA SYSTEMS
MASCHINEN.HEID ST - AF GRUPPEN 'A'
MASCHINEN.HEID ST - ARENDALS FOSSE

### Hong Kong

In [ ]:
equities_HK = pd.read_excel('EQUITIES.xlsx', sheet_name='Hong Kong', index_col='Name')
equities_HK.columns = [c + 'HK' for c in equities_HK.columns]
equities_HK_obx = pd.concat([equities_HK, equities_obx], axis=1)

HK_pairs = CHECK_COINT_2(equities_HK_obx, tag='HK')

  0%|          | 0/1584 [00:00<?, ?it/s]

AIA GROUP - BAKKAFROST
CNOOC - GRIEG SEAFOOD
HONG KONG EXS.& CLEAR. - SCHIBSTED A
ANTA SPORTS PRODUCTS - TOMRA SYSTEMS
CHINA RESOURCES LAND - BAKKAFROST
LONGFOR GROUP HOLDINGS - TOMRA SYSTEMS
LONGFOR GROUP HOLDINGS - NEL
LONGFOR GROUP HOLDINGS - NORWEGIAN AIR SHUTTLE
MTR - EIENDOMSSPAR
SUN HUNG KAI PROPERTIES - TELENOR
HONG KONG AND CHINA GAS - GRIEG SEAFOOD
SUNNY OPTICAL TECH.(GP.) - DNB BANK
SUNNY OPTICAL TECH.(GP.) - ATEA
SUNNY OPTICAL TECH.(GP.) - STOREBRAND
CHINA MENGNIU DAIRY - TOMRA SYSTEMS
CLP HOLDINGS - EIENDOMSSPAR
CHOW TAI FOOK JEWELLERY GP. - NORDIC SEMICONDUCTOR
HENDERSON LD.DEV. - EIENDOMSSPAR
ENN ENERGY HOLDINGS - SALMAR
ENN ENERGY HOLDINGS - TOMRA SYSTEMS
LINK REAL ESTATE INVESTMENT TRUST - GRIEG SEAFOOD
ORIENT OVERSEAS (INTL.) - KONGSBERG GRUPPEN
SEMICON.MNFG.INTL. - NEL
CHINA RESOURCES GAS GROUP - MOWI
CHINA RESOURCES GAS GROUP - BAKKAFROST
CHINA RESOURCES POWER - NORSK HYDRO


### Japan

In [ ]:
equities_Japan = pd.read_excel('EQUITIES.xlsx', sheet_name='Japan', index_col='Name')
equities_Japan.columns = [c + 'Japan' for c in equities_Japan.columns]
equities_Japan_obx = pd.concat([equities_Japan, equities_obx], axis=1)

Japan_pairs = CHECK_COINT_2(equities_Japan_obx, tag='Japan')

  0%|          | 0/2016 [00:00<?, ?it/s]

TOYOTA MOTOR - YARA INTERNATIONAL
NIPPON TELG. & TEL. - AF GRUPPEN 'A'
NIPPON TELG. & TEL. - EIENDOMSSPAR
KDDI - AF GRUPPEN 'A'
SOFTBANK GROUP - NEL
HONDA MOTOR - DNB BANK
HONDA MOTOR - MOWI
HONDA MOTOR - YARA INTERNATIONAL
HONDA MOTOR - GJENSIDIGE FORSIKRING
HONDA MOTOR - ORKLA
HONDA MOTOR - SALMAR
HONDA MOTOR - TOMRA SYSTEMS
HONDA MOTOR - AF GRUPPEN 'A'
HONDA MOTOR - ARENDALS FOSSEKOMPANI
HONDA MOTOR - ATEA
HONDA MOTOR - BAKKAFROST
HONDA MOTOR - BORREGAARD
HONDA MOTOR - KONGSBERG GRUPPEN
HONDA MOTOR - LEROY SEAFOOD GROUP
HONDA MOTOR - VEIDEKKE
MITSUBISHI - DNB BANK
HOYA - GJENSIDIGE FORSIKRING
HOYA - TOMRA SYSTEMS
SUMITOMO MITSUI FINL.GP. - DNB BANK
SUMITOMO MITSUI FINL.GP. - AKER BP
SUMITOMO MITSUI FINL.GP. - MOWI
SUMITOMO MITSUI FINL.GP. - AKER
SUMITOMO MITSUI FINL.GP. - AF GRUPPEN 'A'
SUMITOMO MITSUI FINL.GP. - ATEA
SUMITOMO MITSUI FINL.GP. - AUSTEVOLL SEAFOOD
SUMITOMO MITSUI FINL.GP. - LEROY SEAFOOD GROUP
SUMITOMO MITSUI FINL.GP. - STOREBRAND
FANUC - ORKLA
SMC - GJENSIDIGE FORSIK

### China

In [ ]:
equities_China = pd.read_excel('EQUITIES.xlsx', sheet_name='China', index_col='Name')
equities_China.columns = [c + 'China' for c in equities_China.columns]
equities_China_obx = pd.concat([equities_China, equities_obx], axis=1)

China_pairs = CHECK_COINT_2(equities_China_obx, tag='China')

  0%|          | 0/1620 [00:00<?, ?it/s]

INDUSTRIAL & COML.BK.OF CHINA 'A' - EIENDOMSSPAR
CHINA MERCHANTS BANK 'A' - GJENSIDIGE FORSIKRING
AGRICULTURAL BANK OF CHINA 'A' - EIENDOMSSPAR
WULIANGYE YIBIN 'A' - NEL
PING AN INSURANCE (GP.) CO. OF CHINA 'A' - TELENOR
CHINA YANGTZE PWR. 'A' - AF GRUPPEN 'A'
INDUSTRIAL BANK 'A' - GJENSIDIGE FORSIKRING
INDUSTRIAL BANK 'A' - AF GRUPPEN 'A'
INDUSTRIAL BANK 'A' - VEIDEKKE
HANGZHOU HIK VIS.DIG. TECH.'A' - ATEA
CHINA SHENHUA EN.'A' - DNB BANK
CHINA SHENHUA EN.'A' - YARA INTERNATIONAL
PING AN BANK 'A' - YARA INTERNATIONAL
WANHUA CHEMICAL GP.'A' - TOMRA SYSTEMS
BANK OF NINGBO 'A' - GJENSIDIGE FORSIKRING
JIANGSU YANGHE BREW.JST. 'A' - YARA INTERNATIONAL
BYD 'H' - NORDIC SEMICONDUCTOR
AIER EYE HOSPITAL GP.'A' - SCHIBSTED A
AIER EYE HOSPITAL GP.'A' - NEL
CHINA PAC.IN.(GROUP) 'A' - TELENOR
CHINA VANKE 'A' - ORKLA
CHONGQING ZHIFEI BILG. PRDS.'A' - NEL


### Chile

In [ ]:
equities_Chile = pd.read_excel('EQUITIES.xlsx', sheet_name='Chile', index_col='Name')
equities_Chile.columns = [c + 'Chile' for c in equities_Chile.columns]
equities_Chile_obx = pd.concat([equities_Chile, equities_obx], axis=1)

Chile_pairs = CHECK_COINT_2(equities_Chile_obx, tag='Chile')

  0%|          | 0/1476 [00:00<?, ?it/s]

BICECORP PREF. - TELENOR
COLBUN MACHICURA - SUBSEA 7
EMPRESA NACIONAL DE TELECOMUNICA - GJENSIDIGE FORSIKRING
VINA CONCHA Y TORO - TELENOR
AGRICOLA NACION - YARA INTERNATIONAL


### Sweden

In [ ]:
equities_Sweden = pd.read_excel('EQUITIES.xlsx', sheet_name='Sweden', index_col='Name')
equities_Sweden.columns = [c + 'Sweden' for c in equities_Sweden.columns]
equities_Sweden_obx = pd.concat([equities_Sweden, equities_obx], axis=1)

Sweden_pairs = CHECK_COINT_2(equities_Sweden_obx, tag='Sweden')

  0%|          | 0/1728 [00:00<?, ?it/s]

ASSA ABLOY B - AF GRUPPEN 'A'
SKANDINAVISKA ENSKILDA BANKEN A - YARA INTERNATIONAL
SKANDINAVISKA ENSKILDA BANKEN A - BORREGAARD
TELIA COMPANY - YARA INTERNATIONAL
TELIA COMPANY - SCHIBSTED A
TELIA COMPANY - NORWEGIAN AIR SHUTTLE
SVENSKA HANDELSBANKEN A - DNO
AAK - BAKKAFROST
FASTIGHETS BALDER B - GJENSIDIGE FORSIKRING
LUNDIN ENERGY - SALMAR
SECURITAS B - EIENDOMSSPAR
SWEDISH MATCH - TOMRA SYSTEMS
SWEDISH MATCH - NEL
SWEDISH ORPHAN BIOVITRUM - LEROY SEAFOOD GROUP
SWEDISH ORPHAN BIOVITRUM - NORWAY ROYAL SALMON
TELE2 B - BAKKAFROST
AXFOOD - GJENSIDIGE FORSIKRING
AXFOOD - AF GRUPPEN 'A'
AXFOOD - BAKKAFROST
CASTELLUM - BAKKAFROST
CATENA - TOMRA SYSTEMS
DIOS FASTIGHETER - MOWI
DIOS FASTIGHETER - GJENSIDIGE FORSIKRING
DIOS FASTIGHETER - SALMAR
DIOS FASTIGHETER - BAKKAFROST
FABEGE - MOWI
FABEGE - BAKKAFROST
HUFVUDSTADEN A - EIENDOMSSPAR
HUSQVARNA B - BORREGAARD


### Denmark

In [ ]:
equities_Denmark = pd.read_excel('EQUITIES.xlsx', sheet_name='Denmark', index_col='Name')
equities_Denmark.columns = [c + 'Denmark' for c in equities_Denmark.columns]
equities_Denmark_obx = pd.concat([equities_Denmark, equities_obx], axis=1)

Denmark_pairs = CHECK_COINT_2(equities_Denmark_obx, tag='Denmark')

  0%|          | 0/1620 [00:00<?, ?it/s]

DSV - TOMRA SYSTEMS
NOVOZYMES B - YARA INTERNATIONAL
GN STORE NORD - NEL
DEMANT - DNB BANK
DEMANT - AKER BP
DEMANT - AKER
DEMANT - ATEA
DEMANT - STOREBRAND
ROYAL UNIBREW - TOMRA SYSTEMS
KOBENHAVNS LUFTHAVNE - EIENDOMSSPAR
UTD.INTENT. - DNB BANK
AALBORG BOLDSPILKLUB - EQUINOR
AALBORG BOLDSPILKLUB - DNB BANK
AALBORG BOLDSPILKLUB - TELENOR
AALBORG BOLDSPILKLUB - AKER BP
AALBORG BOLDSPILKLUB - MOWI
AALBORG BOLDSPILKLUB - YARA INTERNATIONAL
AALBORG BOLDSPILKLUB - AKER
AALBORG BOLDSPILKLUB - DNO
AALBORG BOLDSPILKLUB - GJENSIDIGE FORSIKRING
AALBORG BOLDSPILKLUB - GRIEG SEAFOOD
AALBORG BOLDSPILKLUB - ORKLA
AALBORG BOLDSPILKLUB - SALMAR
AALBORG BOLDSPILKLUB - SCHIBSTED A
AALBORG BOLDSPILKLUB - TOMRA SYSTEMS
AALBORG BOLDSPILKLUB - AF GRUPPEN 'A'
AALBORG BOLDSPILKLUB - ARENDALS FOSSEKOMPANI
AALBORG BOLDSPILKLUB - ATEA
AALBORG BOLDSPILKLUB - AUSTEVOLL SEAFOOD
AALBORG BOLDSPILKLUB - BAKKAFROST
AALBORG BOLDSPILKLUB - BONHEUR
AALBORG BOLDSPILKLUB - BORREGAARD
AALBORG BOLDSPILKLUB - EIENDOMSSPAR
AALBO

### Finland

In [ ]:
equities_Finland = pd.read_excel('EQUITIES.xlsx', sheet_name='Finland', index_col='Name')
equities_Finland.columns = [c + 'Finland' for c in equities_Finland.columns]
equities_Finland_obx = pd.concat([equities_Finland, equities_obx], axis=1)

Finland_pairs = CHECK_COINT_2(equities_Finland_obx, tag='Finland')

  0%|          | 0/1620 [00:00<?, ?it/s]

KONE 'B' - NEL
NESTE - NEL
UPM-KYMMENE - BAKKAFROST
STORA ENSO R - STOREBRAND
ELISA - BAKKAFROST
KONECRANES - ATEA
KEMIRA - BAKKAFROST
TIETOEVRY - ORKLA
TIETOEVRY - EIENDOMSSPAR
FISKARS 'A' - YARA INTERNATIONAL
